# Predicción del estrés financiero

# Librerías

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, validation_curve
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from statsmodels.stats.outliers_influence import variance_inflation_factor


%matplotlib inline

# Carga del dataset

In [2]:
#pip install openpyxl

In [3]:
dataset = pd.read_excel('datawork/data/product/filtered.xlsx', engine='openpyxl')

dataset

,Nombre,TD/TA_2021,TD/TA_2022,TD/TA_2023,WC/TA_2021,WC/TA_2022,WC/TA_2023,EBITDA/TA_2021,EBITDA/TA_2022,EBITDA/TA_2023,ln(TA)_2021,ln(TA)_2022,ln(TA)_2023,isFamiliar
0,FELIX SOLIS SOCIEDAD LIMITADA,28.881117,29.229926,32.205115,5.123154e+06,4.178322e+06,5.167578e+06,6.619700e+06,2.545090e+06,7.408183e+06,0.358703,0.336391,0.424802,unknown
1,CODORNIU SA,0.745956,0.985328,0.661631,8.501304e+07,6.806911e+07,1.023132e+08,1.611695e+07,1.099225e+07,2.269676e+07,-0.033204,0.021107,-0.155104,no
2,MIGUEL TORRES SA,1.832514,2.061384,2.266592,9.451695e+07,8.441475e+07,7.487256e+07,8.732763e+06,7.709204e+06,-3.646480e+06,-0.170886,-0.253326,-0.215693,unknown
3,GARCIA CARRION 1890 SL,3.650187,3.568547,8.457106,1.063484e+07,2.095649e+08,1.732161e+08,3.527727e+06,1.389730e+08,1.977129e+08,-0.104308,-1.393215,-1.703137,yes
4,"BARON DE LEY, S.A.",110.334803,9.072074,3.148995,8.313455e+07,1.875481e+08,9.133677e+07,1.642012e+07,6.938583e+07,3.007278e+07,0.333328,-0.655692,0.247142,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,BODEGAS DEL SEÑORIO SL.,0.186035,0.330242,0.421280,3.296214e+05,3.159593e+05,2.720337e+05,-3.779646e+04,-1.625587e+05,-4.818291e+04,0.846086,0.459976,0.446193,unknown
1353,CEPA SUBBETICA SL.,180.315759,211.118315,368.166239,-8.981009e+05,-1.037088e+06,-1.793044e+06,-2.269088e+05,-1.827665e+05,-2.958767e+05,-1.130543,-1.072313,-1.439112,unknown
1354,ENOMAR SL,1.593659,1.660433,1.823852,-2.292044e+05,-2.433999e+05,-2.866429e+05,-2.553964e+04,1.633185e+02,2.408455e+03,-0.036076,-0.052600,-0.100445,unknown
1355,UNION DE VITICULTORES RIOJANOS S.L.,151.924806,112.457839,62.418529,7.260560e+05,1.033120e+06,1.532601e+06,-2.037728e+03,9.697731e+04,-4.941207e+03,-1.345299,-1.596186,-1.768185,unknown


# Semilla

In [4]:
myseed=1234

# Métricas

In [5]:
from sklearn.metrics import make_scorer
from sklearn.metrics import average_precision_score, recall_score

recall_scorer = make_scorer(recall_score, pos_label=1,labels=[0,1])
spec_scorer = make_scorer(recall_score, pos_label=0,labels=[0,1])
pr_auc_scorer = make_scorer(average_precision_score)

accuracy = []
f1 = []
precision = []
recall = []
spec=[]
roc_auc = []
prec_rec = []

dicci = {"accuracy":"accuracy", "average_precision":pr_auc_scorer,
         "f1":"f1", "precision":"precision", "recall":recall_scorer,
         "spec":spec_scorer,"roc_auc":"roc_auc"}

def score(scores):
    accuracy.append(str(np.around(np.mean(scores.get("test_accuracy")),3)) + " ("
                + str(np.around(np.std(scores.get("test_accuracy")), 3))+ ")")
    f1.append(str(np.around(np.mean(scores.get("test_f1")),3)) + " ("
                + str(np.around(np.std(scores.get("test_f1")), 3))+ ")")
    precision.append(str(np.around(np.mean(scores.get("test_precision")),3)) + " ("
                + str(np.around(np.std(scores.get("test_precision")), 3))+ ")")
    recall.append(str(np.around(np.mean(scores.get("test_recall")),3)) + " ("
                + str(np.around(np.std(scores.get("test_recall")), 3))+ ")")
    spec.append(str(np.around(np.mean(scores.get("test_spec")),3)) + " ("
                + str(np.around(np.std(scores.get("test_spec")), 3))+ ")")
    roc_auc.append(str(np.around(np.mean(scores.get("test_roc_auc")),3)) + " ("
                + str(np.around(np.std(scores.get("test_roc_auc")), 3))+ ")")

# Regresión logística

Los algoritmos ya tienen integrados la validación cruzada estratificada, una vez limpien los datos y elaboren las variables. Prueben con este para empezar

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

#solvers = ['liblinear','lbfgs','saga','sag']
#penal = ['l1','l2','elasticnet']
logistic = LogisticRegressionCV(random_state=myseed, max_iter=200, solver='liblinear',penalty='l1')
scaler = StandardScaler()
scores = cross_validate(logistic, X,Y, cv=10, scoring=dicci)
score(scores)
scores

NameError: name 'X' is not defined

In [ ]:
from sklearn.model_selection import cross_val_predict
logistic = LogisticRegressionCV(random_state=myseed, max_iter=200, solver='liblinear',penalty='l1')
scaler = StandardScaler()
prediccioneslogistic = cross_val_predict(logistic, X,Y, cv=10)
prediccioneslogisticproba = cross_val_predict(logistic, X,Y, cv=10, method="predict_proba")

# Random Forest

La idea básica del bagging es utilizar el conjunto de entrenamiento original para generar centenares o miles de conjuntos similares usando muestreo con reemplazo. En este concepto está basado el algoritmo Random Forest, la combinación de varios árboles de decisión, cada uno entrenado con una realización diferente de los datos. La decisión final del clasificador combinado (el Random Forest) se toma por mayoría, dando el mismo peso a todas las decisiones parciales tomadas por los clasificadores base (los árboles).

In [ ]:
#depth = [10,25,50,75,100,125]
#estimadores= [1000]

rfc = RandomForestClassifier(random_state=myseed, max_depth=32, n_estimators=954)
scoresrfc = cross_validate(rfc, X, Y, cv=10, scoring=dicci)
score(scoresrfc)
scoresrfc

In [ ]:
rfc = RandomForestClassifier(random_state=myseed, max_depth=32, n_estimators=954)
prediccionesrfc = cross_val_predict(rfc, X, Y, cv=10)
prediccionesrfcproba = cross_val_predict(rfc, X,Y, cv=10, method="predict_proba")

# KNN

In [ ]:
#algoritmo = ['auto','ball_tree','kd_tree','brute']
#p = [1,2]
#metric = ['euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean','mahalanobis']

neigh = KNeighborsClassifier(n_neighbors=2, algorithm='auto',p=1, weights="distance")
scoresneigh = cross_validate(neigh, X, Y, cv=10, scoring=dicci)
score(scoresneigh)
scoresneigh
print(str(np.around(np.mean(scoresneigh['test_roc_auc']),3)))

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=2, algorithm='auto',p=1, weights="distance")
prediccionesneigh = cross_val_predict(neigh, X, Y, cv=10)
prediccionesneighproba = cross_val_predict(neigh, X,Y, cv=10, method="predict_proba")

# SVM

In [ ]:
from sklearn.model_selection import cross_validate
#kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
#C = [1.0,5.0,10.0,50.0,100.0]
#gammas = [0.1,0.01,0.05,0.001,0.005]

svm = SVC(gamma=0.0023420730885665587,probability=True, C=55.56346718268796)
scoressvm = cross_validate(svm, X, Y, cv=10, scoring=dicci)
score(scoressvm)
print(str(np.around(np.mean(scoressvm['test_f1']),3)))
print(str(np.around(np.mean(scoressvm['test_spec']),3)))
print(str(np.around(np.mean(scoressvm['test_roc_auc']),3)))

In [ ]:
svm = SVC(gamma=0.0023420730885665587,probability=True, C=55.56346718268796)
prediccionessvm = cross_val_predict(svm, X, Y, cv=10)
prediccionessvmproba = cross_val_predict(svm, X, Y, cv=10, method="predict_proba")

# Resultados

In [ ]:
conjunto1 = pd.DataFrame(columns=["accuracy", "f1_score", "precision","recall","specificity","roc_auc"])
conjunto1["accuracy"] = accuracy
conjunto1["f1_score"] = f1
conjunto1["precision"] = precision
conjunto1["recall"] = recall
conjunto1["specificity"] = spec
conjunto1["roc_auc"] = roc_auc
conjunto1.index = ["LOGIT","RFC", "KNN","SVM"]
conjunto1.to_csv("conjunto1.csv")

In [ ]:
conjunto1

In [ ]:
conprobas = []

conprobas.append(prediccioneslogisticproba)
conprobas.append(prediccionesneighproba)
conprobas.append(prediccionesrfcproba)
conprobas.append(prediccionessvmproba)

In [ ]:
clasificadores = ["LOGIT","RFC", "KNN","SVM"]
colores = ["brown","coral","gold","green"]

In [ ]:
#roc_auc[0] = '0.805 (0.018)'
#roc_auc[1] = '0.705 (0.038)'
#roc_auc[2] = '0.901 (0.023)'
#roc_auc[3] = '0.890 (0.018)'
#roc_auc[4] = '0.976 (0.008)'
#roc_auc[5] = '0.972 (0.015)'

In [ ]:
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from scipy import interp

def plot_roc3(y_true, probis,testnn, colores,nombres,title='ROC Curves',
                   plot_micro=True, plot_macro=True, classes_to_plot=None,
                   ax=None, figsize=None, cmap='nipy_spectral',
                   title_fontsize="large", text_fontsize="medium"):

    testnn = np.array(testnn)
    y_true = np.array(y_true)
    classes = np.unique(y_true)

    for j in range(6):
        probis[j] = np.array(probis[j])
        probas = probis[j]
        if classes_to_plot is None:
            classes_to_plot = classes
        if ax is None:
            fig, ax = plt.subplots(1, 1, figsize=figsize)
            ax.set_title(title, fontsize=title_fontsize)
        fpr_dict = dict()
        tpr_dict = dict()
        indices_to_plot = np.in1d(classes, classes_to_plot)

        if j ==0:
            for i, to_plot in enumerate(indices_to_plot):
                fpr_dict[i], tpr_dict[i], _ = roc_curve(testnn, probas[:, i],
                                                pos_label=classes[i])

            if plot_macro:
                ax.plot(fpr_dict[0], tpr_dict[0], label='ROC Curve - ' + nombres[j] + ' - ' + roc_auc[j], color=colores[j], linestyle=':', linewidth=4)
        else:
            for i, to_plot in enumerate(indices_to_plot):
                fpr_dict[i], tpr_dict[i], _ = roc_curve(y_true, probas[:, i],
                                                pos_label=classes[i])
            if plot_macro:
                ax.plot(fpr_dict[0], tpr_dict[0], label='ROC Curve - ' + nombres[j] + ' - ' + roc_auc[j], color=colores[j], linestyle=':', linewidth=4)

            ax.plot([0, 1], [0, 1], 'k--', lw=2)
            ax.set_xlim([0.0, 1.0])
            ax.set_ylim([0.0, 1.05])
            ax.set_xlabel('False Positive Rate', fontsize=text_fontsize)
            ax.set_ylabel('True Positive Rate', fontsize=text_fontsize)
            ax.tick_params(labelsize=text_fontsize)
            ax.legend(loc='lower right', fontsize=text_fontsize)
    return ax

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
plot_roc3(Y, conprobas,TEST2, colores = colores, nombres = clasificadores, title="Multi-classifiers - ROC Curves" )
plt.show()